In [18]:
import pandas as pd
from collections import defaultdict,namedtuple
from itertools import combinations


In [19]:
class Apriori:
    def __init__(self, transactions, min_support):
        self.transactions = transactions
        self.min_support = min_support
        self.itemsets = []

    def find_frequent_itemsets(self):
        itemsets = self.find_frequent_1_itemsets()
        k = 2
        while itemsets:
            self.itemsets.extend(itemsets)
            itemsets = self.generate_candidates(itemsets, k)
            itemsets = self.filter_candidates(itemsets, k)
            k += 1
        return self.itemsets

    def find_frequent_1_itemsets(self):
        item_counts = defaultdict(int)
        for transaction in self.transactions:
            for item in transaction:
                item_counts[item] += 1
        return [frozenset([item]) for item, count in item_counts.items() if count >= self.min_support]

    def generate_candidates(self, prev_itemsets, k):
        candidates = set()
        for itemset1 in prev_itemsets:
            for itemset2 in prev_itemsets:
                union = itemset1.union(itemset2)
                if len(union) == k:
                    candidates.add(union)
        return candidates

    def filter_candidates(self, candidates, k):
        frequent_itemsets = []
        item_counts = defaultdict(int)
        for transaction in self.transactions:
            for candidate in candidates:
                if candidate.issubset(transaction):
                    item_counts[candidate] += 1
        for itemset, count in item_counts.items():
            if count >= self.min_support * k:
                frequent_itemsets.append(itemset)
        return frequent_itemsets

class Node:
  """
  A node in the FP-tree.
  """
  def __init__(self, item, count, parent=None):
    self.item = item
    self.count = count
    self.parent = parent
    self.children = {}

def build_fptree(data, frequent_items):
  """
  Builds the FP-tree from the transaction data.

  Args:
      data: Transaction data as a pandas DataFrame.
      frequent_items: Frequent items for filtering transactions.

  Returns:
      A tuple containing the root node of the FP-tree and the header table.
  """
  fp_tree = Node(None, 1)
  header_table = {}
  for transaction in data.itertuples():
    transaction = [item for item in transaction[1:] if item in frequent_items]
    transaction.sort(key=lambda x: frequent_items[x], reverse=True)
    update_tree(fp_tree, transaction, header_table)
  return fp_tree, header_table

def update_tree(node, transaction, header_table):
  """
  Updates the FP-tree with a new transaction.

  Args:
      node: The current node in the FP-tree.
      transaction: A list of items in the transaction.
      header_table: A dictionary storing frequent item information.
  """
  if not transaction:
    return

  # Find the first element in the transaction
  item = transaction[0]
  child = node.children.get(item)

  # If the item exists as a child, increment its count
  if child:
    child.count += 1
  else:
    # Create a new child node
    child = Node(item, 1, node)
    node.children[item] = child

    # Update header table
    if item in header_table:
      current = header_table[item]
      while current.parent is not None:
        current = current.parent
      current.next = child
    else:
      header_table[item] = child

  # Recursively update the tree with the remaining transaction items
  update_tree(child, transaction[1:], header_table)

In [20]:
# Load dataset using pd.read_csv
df = pd.read_csv('store.csv')  # Replace 'your_dataset.csv' with your dataset file path
transactions = df.apply(set, axis=1).tolist()

In [21]:
min_support = 2

In [16]:
# Apriori
apriori = Apriori(transactions, min_support)
print("Frequent itemsets using Apriori:")
print(apriori.find_frequent_itemsets())


Frequent itemsets using Apriori:
[frozenset({'burgers'}), frozenset({nan}), frozenset({'eggs'}), frozenset({'meatballs'}), frozenset({'chutney'}), frozenset({'avocado'}), frozenset({'turkey'}), frozenset({'whole wheat rice'}), frozenset({'energy bar'}), frozenset({'green tea'}), frozenset({'mineral water'}), frozenset({'milk'}), frozenset({'low fat yogurt'}), frozenset({'french fries'}), frozenset({'whole wheat pasta'}), frozenset({'light cream'}), frozenset({'shallot'}), frozenset({'soup'}), frozenset({'spaghetti'}), frozenset({'frozen vegetables'}), frozenset({'pet food'}), frozenset({'cookies'}), frozenset({'cooking oil'}), frozenset({'champagne'}), frozenset({'salmon'}), frozenset({'chicken'}), frozenset({'oil'}), frozenset({'honey'}), frozenset({'chocolate'}), frozenset({'shrimp'}), frozenset({'black tea'}), frozenset({'extra dark chocolate'}), frozenset({'fresh tuna'}), frozenset({'tomatoes'}), frozenset({'protein bar'}), frozenset({'red wine'}), frozenset({'pasta'}), frozenset({

In [22]:
# FP Growth
fp_growth = FP_Growth(transactions, min_support)
print("\nFrequent itemsets using FP Growth:")
print(fp_growth.find_frequent_itemsets())


Frequent itemsets using FP Growth:


AttributeError: 'TreeNode' object has no attribute 'link'